In [5]:
import os
import pandas as pd
import pandas_profiling as pf

In [18]:
#Initializing file paths
raw_dir = os.path.join(os.pardir,os.pardir,'data','raw')
processed_dir = os.path.join(os.pardir,os.pardir,'data','processed')

In [19]:
data_path = os.path.join(raw_dir,'dev.csv')

In [22]:
data_df = pd.read_csv(data_path)

In [23]:
data_df.head()

,ex_id,user_id,prod_id,rating,label,date,review
0,11,934,0,5.0,1,2014-01-20,"all around good place, cozy, I came in and did..."
1,17,940,0,4.0,0,2014-09-16,"For lunch, my friend and I had: -Lamb sandwich..."
2,20,943,0,5.0,0,2014-05-24,Some good Big Greek cooking!! Came to City on ...
3,30,953,0,4.0,0,2013-10-17,So... as you may notice from some of my other ...
4,43,966,0,3.0,0,2012-12-19,"I don't understand the whole ""You can't order ..."


#### Summary Statistics

In [24]:
# A quick summary statistics on all data.
data_df.describe()

,ex_id,user_id,prod_id,rating,label
count,35918.000000,35918.000000,35918.000000,35918.000000,35918.000000
mean,179060.247035,53625.825853,457.526811,4.023275,0.101565
std,103651.452541,45583.076589,260.532673,1.055449,0.302079
min,11.000000,923.000000,0.000000,1.000000,0.000000
25%,89904.000000,13781.500000,247.000000,4.000000,0.000000
50%,178631.500000,40110.000000,465.000000,4.000000,0.000000
75%,268548.750000,86310.000000,668.000000,5.000000,0.000000
max,358935.000000,161140.000000,922.000000,5.000000,1.000000


In [42]:
# Checking the number users in the data
data_df.user_id.count()

35918

In [41]:
# Checking the number of unique users in the data
len(data_df.user_id.unique())

28040

*_Assumption 1: Since unique user count is less than user count we can assume that some users wrote multiple reviews or thesame review by thesame user was added multiple times. (We'll confirm which one later)_*

In [45]:
# Checking the number ex_id in the data
data_df.ex_id.count()

35918

In [44]:
len(data_df.ex_id.unique())

35918

*_Question 1: What's `ex_id`, hmmm? (Could it be a unique id for each review since it's value is unique for each review)_*

Since the df.describe() is best suited for numeric features its best to call it after transforming the review words into to a bag of words representation.

In [31]:
# Seperating the actual reviews and labels in a separate dataframe

review_df = data_df[['review','label']].copy()
review_df.head()

,review,label
0,"all around good place, cozy, I came in and did...",1
1,"For lunch, my friend and I had: -Lamb sandwich...",0
2,Some good Big Greek cooking!! Came to City on ...,0
3,So... as you may notice from some of my other ...,0
4,"I don't understand the whole ""You can't order ...",0


In [51]:
import nltk

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [53]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [ ]:
stop_words = set(stopwords.words('english')) 

In [ ]:
review_df['tokens'] = review

In [56]:
len(stopwords.words('english'))

179

In [57]:
len(set(stopwords.words('english')) )

179